In [1]:
from dotenv import load_dotenv
from crewai import Agent,LLM, Task, Crew, Process
from crewai.tools import tool
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
import openlit
import os
from pydantic import BaseModel, Field
from typing import List, Optional
from tavily import TavilyClient
from scrapegraph_py import Client
import json
load_dotenv()

True

In [2]:
os.environ["OPEN_LIT_ENDPOINT"] = os.getenv("OPEN_LIT_ENDPOINT")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GROQ_MODEL_NAME"] = os.getenv("GROQ_MODEL_NAME")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["SCRAPE_GRAPH_AI_KEY"] = os.getenv("SCRAPE_GRAPH_AI_KEY")

In [3]:
no_keywords = 10
about_company = "Buyout is a company that provides AI solutions to help websites refine their search and recommendation systems."

company_context = StringKnowledgeSource(
    content=about_company
)

In [4]:
openlit.init(otlp_endpoint=os.environ["OPEN_LIT_ENDPOINT"],disable_metrics=False)

Overriding of current TracerProvider is not allowed


In [5]:
base_llm = LLM(
    api_key=os.environ["GROQ_API_KEY"],
    model=os.environ["GROQ_MODEL_NAME"],
    temperature=0.1,
)
search_client = TavilyClient(api_key = os.environ["TAVILY_API_KEY"])
scrape_client = Client(api_key = os.environ["SCRAPE_GRAPH_AI_KEY"])

In [6]:
#search_client.search("coffe machine site:amazon.eg")

In [7]:
output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)

In [8]:
class SuggestedSearchQueries(BaseModel):
    queries: List[str] = Field(...,title="Suggested search queries to be passed to the search engine",min_items =1 , max_items=no_keywords)

search_query_agent = Agent(
    role = "Search Queries Recommendation Agent",
    goal = "\n".join(["To provide a list of suggested search queries to be passed to the search engine.",
            "The queries must be varied and looking for specific items.",
            ]),
    backstory = "The agent is designed to help in looking for products by providing a list of suggested search queries to be passed to the search engine based on the context provided",
    llm = base_llm,
    verbose = True
)
search_query_task = Task(
    description = "\n".join([
        "{company_name} is looking to buy {product_name} at the best prices (value for a price strategy)",
        "The company target any of these websites to buy from: {websites_list}",
        "The company wants to reach all available proucts on the internet to be compared later in another stage.",
        "The stores must sell the product in {country_name}",
        "Generate at maximum {no_keywords} queries.",
        "The search keywords must be in {language} language.",
        "Search keywords must contains specific brands, types or technologies. Avoid general keywords.",
        "The search query must reach an ecommerce webpage for product, and not a blog or listing page."
    ]),
    expected_output = "A JSON object containing a list of the suggested search queries.",
    output_json = SuggestedSearchQueries,
    output_file = os.path.join(output_dir,"step_1_suggested_search_queries.json"),
    agent = search_query_agent
)


Run the AI Crew

In [9]:
# company_crew = Crew(
#     agents = [search_query_agent],
#     tasks = [search_query_task],
#     process = Process.sequential
# )

In [10]:
# crew_results = company_crew.kickoff(inputs = {
#     "company_name":"Buyout",
#     "product_name":"coffee machine for the office",
#     "websites_list": ["www.amazon.eg","www.jumia.com.eg","www.noon.com/egypt-en"],
#     "country_name":"Egypt",
#     "no_keywords":10,
#     "language":"English"
# })

Agent B

In [11]:
class SingleSearchResult(BaseModel):
    title: str
    url: str = Field(...,title='the page url')
    content: str
    score: float
    search_query: str

class AllSearchResults(BaseModel):
    results: List[SingleSearchResult]

@tool
def search_engine_tool(query: str) -> str:
    '''This is Useful for search based queries. Use this to find the current information about any query related to pages using a search engine'''
    return search_client.search(query)

search_engine_agent = Agent(
    role = 'Search Engine Agent',
    goal = 'To search for products based on the suggested search query',
    backstory = 'The agent is designed to help in looking for products by searching for products based on the suggested search queries.',
    llm = base_llm,
    verbose = True,
    tools = [search_engine_tool]
)
search_engine_task = Task(
    description = '\n'.join([
        'The task is to search for products based on the suggested search queries.',
        'You have to collect results from multiple search queries.',
        'Ignore any susbicious links or not an ecommerce single product website link.',
        'Ignore any search results with confidence score less than ({score_th}) .',
        'The search results will be used to compare prices of products from different websites.'
    ]),
    expected_output = 'A JSON containing the search results',
    output_json=AllSearchResults,
    output_file = os.path.join(output_dir, "step_2_search_results.json"),
    agent = search_engine_agent
)


In [12]:
# company_crew = Crew(
#     agents = [search_query_agent,search_engine_agent],
#     tasks = [search_query_task, search_engine_task],
#     process = Process.sequential
# )

In [13]:
# crew_results = company_crew.kickoff(inputs = {
#     "company_name":"Buyout",
#     "product_name":"coffee machine for the office",
#     "websites_list": ["www.amazon.eg","www.jumia.com.eg","www.noon.com/egypt-en"],
#     "country_name":"Egypt",
#     "no_keywords":10,
#     "language":"English",
#     "score_th": 0.40
# })

Agent C

In [14]:
class ProductSpec(BaseModel):
    specification_name: str
    specification_value: str

class SingleExtractedProduct(BaseModel):
    page_url: str = Field(..., title="The original url of the product page")
    product_title: str = Field(..., title="The title of the product")
    product_image_url: str = Field(..., title="The url of the product image")
    product_url: str = Field(..., title="The url of the product")
    product_current_price: float = Field(..., title="The current price of the product")
    product_original_price: float = Field(title="The original price of the product before discount. Set to None if no discount", default=None)
    product_discount_percentage: float = Field(title="The discount percentage of the product. Set to None if no discount", default=None)

    product_specs: List[ProductSpec] = Field(..., title="The specifications of the product. Focus on the most important specs to compare.", min_items=1, max_items=5)

    agent_recommendation_rank: int = Field(..., title="The rank of the product to be considered in the final procurement report. (out of 5, Higher is Better) in the recommendation list ordering from the best to the worst")
    agent_recommendation_notes: List[str]  = Field(..., title="A set of notes why would you recommend or not recommend this product to the company, compared to other products.")

class AllExtractedProducts(BaseModel):
    products: List[SingleExtractedProduct]

@tool
def web_scraping_tool(page_url: str):
    '''An AI tool to help an agent to scrape a web page.
    Example:
    
    web_scraping_tool(
        page_url = 'https://www.amazon.com/dp/B0FBSP99LX/ref=sspa_dk_detail_right_aax_0?sp_csd=d2lkZ2V0TmFtZT1zcF9kZXRhaWxfcmlnaHRfc2hhcmVk&th=1',
    )
    '''
    details = scrape_client.smartscraper(
        website_url = page_url,
        user_prompt = ''.join(["Extract ''' JSON\n" + SingleExtractedProduct.schema_json() + " '''\n From the web page. ",
                                "Return only the result for the main product on the page, not any other suggested or related products.",
                                "Return the prices in the local currency as shown on the page."]),
    )
    return {
        "page_url":page_url,
        "details": details
    }

In [15]:
# page_url = 'https://www.amazon.com/dp/B0FBSP99LX/ref=sspa_dk_detail_right_aax_0?sp_csd=d2lkZ2V0TmFtZT1zcF9kZXRhaWxfcmlnaHRfc2hhcmVk&th=1'
# required_fields = ['product_title','current_price','old_price']
# page_details = web_scraping_tool(page_url = page_url, required_fields = required_fields)
# page_details

In [23]:
scraping_agent = Agent(
    role = 'Web scraping agent',
    goal = 'To extract details from any website',
    backstory = '\n'.join(['The agent is designed to help in looking for required values from any website url.',
                'These details will be used to decide which best product to buy.']),
    llm = base_llm,
    tools = [web_scraping_tool],
    verbose = True
)
scraping_task = Task(
    description = '\n'.join([
        "The task is to extract product details from any ecommerce store page url.",
        "The task has to collect results from multiple pages urls",
    ]),
    expected_output = "A JSON object containing products details",
    output_json = AllExtractedProducts,
    output_file = os.path.join(output_dir, 'step_3_search_results.json'),
    agent = scraping_agent
)

Agent D

In [24]:
html_report_agent = Agent(
    role="HTML Report Generator Agent",
    goal="To generate a professional HTML page for the procurement report",
    backstory="The agent is designed to assist in generating a professional HTML page for the procurement report after looking into a list of products.",
    llm = base_llm,
    verbose = True
)

html_report_task = Task(
    description="\n".join([
        "The task is to generate a professional HTML page for the procurement report.",
        "You have to use Bootstrap CSS framework for a better UI.",
        "Use the provided context about the company to make a specialized report.",
        "The report will include the search results and prices of products from different websites.",
        "The report should be structured with the following sections:",
        "1. Executive Summary: A brief overview of the procurement process and key findings.",
        "2. Introduction: An introduction to the purpose and scope of the report.",
        "3. Methodology: A description of the methods used to gather and compare prices.",
        "4. Findings: Detailed comparison of prices from different websites, including tables and charts.",
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations.",
        "6. Recommendations: Suggestions for procurement based on the analysis.",
        "7. Conclusion: A summary of the report and final thoughts.",
        "8. Appendices: Any additional information, such as raw data or supplementary materials.",
    ]),

    expected_output="A professional HTML page for the procurement report.",
    output_file=os.path.join(output_dir, "step_4_procurement_report.html"),
    agent=html_report_agent,
)

In [25]:
company_crew = Crew(
    agents = [search_query_agent,search_engine_agent,scraping_agent,html_report_agent],
    tasks = [search_query_task, search_engine_task, scraping_task, html_report_task],
    process = Process.sequential
)

In [26]:
crew_results = company_crew.kickoff(inputs = {
    "company_name":"Buyout",
    "product_name":"coffee machine for the office",
    "websites_list": ["www.amazon.eg","www.jumia.com.eg","www.noon.com/egypt-en"],
    "country_name":"Egypt",
    "no_keywords":10,
    "language":"English",
    "score_th": 0.5,
    #"top_recommendations_no":10,
})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Search Queries Recommendation Agent                                                                     │
│                                                                                                                 │
│  Task: Buyout is looking to buy coffee machine for the office at the best prices (value for a price strategy)   │
│  The company target any of these websites to buy from: ['www.amazon.eg', 'www.jumia.com.eg',                    │
│  'www.noon.com/egypt-en']                                                                                       │
│  The company wants to reach all available proucts on the internet to be compared later in another stage.        │
│  The stores must sell the product in Egypt                                                                      │
│  Generate at maximum 10 queries.                                                                                │
│  The search keywords must be in English language.                                                               │
│  Search keywords must contains specific brands, types or technologies. Avoid general keywords.                  │
│  The search query must reach an ecommerce webpage for product, and not a blog or listing page.                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Search Queries Recommendation Agent                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "queries": [                                                                                                 │
│      "buy De'Longhi coffee machine on www.amazon.eg",                                                           │
│      "purchase Nespresso coffee machine on www.jumia.com.eg",                                                   │
│      "price of Siemens coffee machine on www.noon.com/egypt-en",                                                │
│      "Bosch Tassimo coffee machine for sale on www.amazon.eg",                                                  │
│      "Jura coffee machine shop on www.jumia.com.eg",                                                            │
│      "Melitta coffee machine deals on www.noon.com/egypt-en",                                                   │
│      "buy Philips coffee machine online on www.amazon.eg",                                                      │
│      "Capresso coffee machine store on www.jumia.com.eg",                                                       │
│      "Krups coffee machine for sale in Egypt on www.noon.com/egypt-en",                                         │
│      "buy Smeg coffee machine on www.amazon.eg"                                                                 │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Search Engine Agent                                                                                     │
│                                                                                                                 │
│  Task: The task is to search for products based on the suggested search queries.                                │
│  You have to collect results from multiple search queries.                                                      │
│  Ignore any susbicious links or not an ecommerce single product website link.                                   │
│  Ignore any search results with confidence score less than (0.5) .                                              │
│  The search results will be used to compare prices of products from different websites.                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Search Engine Agent                                                                                     │
│                                                                                                                 │
│  Thought: Thought: I need to iterate over each search query and use the search_engine_tool to find relevant     │
│  products, then filter out suspicious links and results with low confidence scores.                             │
│                                                                                                                 │
│  Using Tool: search_engine_tool                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"buy De'Longhi coffee machine on www.amazon.eg\"}"                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web scraping agent                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "products": [                                                                                                │
│      {                                                                                                          │
│        "page_url": "https://www.amazon.eg/-/en/coffee-maker-delonghi/s?k=coffee+maker+delonghi",                │
│        "product_title": "De'Longhi Dedica Style Pump Espresso Machine coffee maker, Silver, EC685M -            │
│  International Version",                                                                                        │
│        "product_image_url": "https://m.media-amazon.com/images/I/61ZOKfowwYL._AC_UL320_.jpg",                   │
│        "product_url":                                                                                           │
│  "https://www.amazon.eg/-/en/DeLonghi-Dedica-Espresso-Machine-coffee/dp/B089P1ZSTW/ref=sr_1_1?",                │
│        "product_current_price": 7849.0,                                                                         │
│        "product_original_price": 14499.0,                                                                       │
│        "product_discount_percentage": 45.93,                                                                    │
│        "product_specs": [                                                                                       │
│          {                                                                                                      │
│            "specification_name": "Brand",                                                                       │
│            "specification_value": "De'Longhi"                                                                   │
│          },                                                                                                     │
│          {                                                                                                      │
│            "specification_name": "Model",                                                                       │
│            "specification_value": "EC685M"                                                                      │
│          },                                                                                                     │
│          {                                                                                                      │
│            "specification_name": "Type",                                                                        │
│            "specification_value": "Espresso Machine"                                                            │
│          },                                                                                                     │
│          {                                                                                                      │
│            "specification_name": "Color",                                                                       │
│            "specification_value": "Silver"                                                                      │
│          },                                                                                                     │
│          {                                             

Failed to convert text into JSON, error: <failed_attempts>

<generation number="1">
<exception>
    litellm.BadRequestError: GroqException - {"error":{"message":"Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.","type":"invalid_request_error","code":"tool_use_failed","failed_generation":"\u003cfunction=AllExtractedProducts\u003e{\"products\": [{\"page_url\": \"https://www.amazon.eg/-/en/coffee-maker-delonghi/s?k=coffee+maker+delonghi\", \"product_title\": \"De'Longhi Dedica Style Pump Espresso Machine coffee maker, Silver, EC685M - International Version\", \"product_image_url\": \"https://m.media-amazon.com/images/I/61ZOKfowwYL._AC_UL320_.jpg\", \"product_url\": \"https://www.amazon.eg/-/en/DeLonghi-Dedica-Espresso-Machine-coffee/dp/B089P1ZSTW/ref=sr_1_1?\", \"product_current_price\": 7849.0, \"product_original_price\": 14499.0, \"product_discount_percentage\": 45.93, \"product_specs\": [{\"specification_name\": \"Brand\", \"specification_

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: HTML Report Generator Agent                                                                             │
│                                                                                                                 │
│  Task: The task is to generate a professional HTML page for the procurement report.                             │
│  You have to use Bootstrap CSS framework for a better UI.                                                       │
│  Use the provided context about the company to make a specialized report.                                       │
│  The report will include the search results and prices of products from different websites.                     │
│  The report should be structured with the following sections:                                                   │
│  1. Executive Summary: A brief overview of the procurement process and key findings.                            │
│  2. Introduction: An introduction to the purpose and scope of the report.                                       │
│  3. Methodology: A description of the methods used to gather and compare prices.                                │
│  4. Findings: Detailed comparison of prices from different websites, including tables and charts.               │
│  5. Analysis: An analysis of the findings, highlighting any significant trends or observations.                 │
│  6. Recommendations: Suggestions for procurement based on the analysis.                                         │
│  7. Conclusion: A summary of the report and final thoughts.                                                     │
│  8. Appendices: Any additional information, such as raw data or supplementary materials.                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: HTML Report Generator Agent                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```html                                                                                                        │
│  <!DOCTYPE html>                                                                                                │
│  <html lang="en">                                                                                               │
│  <head>                                                                                                         │
│      <meta charset="UTF-8">                                                                                     │
│      <meta name="viewport" content="width=device-width, initial-scale=1.0">                                     │
│      <title>Procurement Report</title>                                                                          │
│      <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css">       │
│  </head>                                                                                                        │
│  <body>                                                                                                         │
│      <div class="container">                                                                                    │
│          <h1 class="text-center">Procurement Report</h1>                                                        │
│          <hr>                                                                                                   │
│          <h2>Executive Summary</h2>                                                                             │
│          <p>This report provides an overview of the procurement process for coffee machines from various        │
│  online retailers. The report includes a comparison of prices, product specifications, and recommendations for  │
│  procurement.</p>                                                                                               │
│          <h2>Introduction</h2>                                                                                  │
│          <p>The purpose of this report is to provide a comprehensive analysis of the procurement process for    │
│  coffee machines. The scope of the report includes a review of online retailers, product specifications, and    │
│  prices.</p>                                                                                                    │
│          <h2>Methodology</h2>                                                                                   │
│          <p>The methodology used in this report included a review of online retailers, product specifications,  │
│  and prices. The data was collected from various online sources, including Amazon, Jumia, and Noon.</p>         │
│          <h2>Findings</h2>                                                                                      │
│          <table class="table table-striped">                                                                    │
│              <thead>                                                                                            │
│                  <tr>                                                                                           │
│                      <th>Product</th>                                                                           │
│                      <th>Price</th>                    

╭─────────────────────────────────────────── Trace Batch Finalization ────────────────────────────────────────────╮
│ ✅ Trace batch finalized with session ID: 68db1984-ba22-42c4-8adf-18a451838f84                                  │
│                                                                                                                 │
│ 🔗 View here:                                                                                                   │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/68db1984-ba22-42c4-8adf-18a451838f84?access_code=TRA │
│ CE-5d0cb3d82f                                                                                                   │
│ 🔑 Access Code: TRACE-5d0cb3d82f                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯